<a href="https://colab.research.google.com/github/AISaturdaysLagos/Cohort8-Ransome-Kuti-Adai/blob/main/Copy_of_D_proj_churnrate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Credit card applications
<p>Commercial banks receive <em>a lot</em> of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this notebook, we will build an automatic credit card approval predictor using machine learning techniques, just like the real banks do.</p>
<p><img src="https://assets.datacamp.com/production/project_558/img/credit_card.jpg" alt="Credit card being held in hand"></p>
<p>I'll use the <a href="http://archive.ics.uci.edu/ml/datasets/credit+approval">Credit Card Approval dataset</a> from the UCI Machine Learning Repository. The structure of this notebook is as follows:</p>
<ul>
<li>First, I will start off by loading and viewing the dataset.</li>
<li>I will see that the dataset has a mixture of both numerical and non-numerical features, that it contains values from different ranges, plus that it contains a number of missing entries.</li>
<li>I will have to preprocess the dataset to ensure the machine learning model we choose can make good predictions.</li>
<li>After the data is in good shape, I will do some exploratory data analysis to build our intuitions.</li>
<li>Finally, I will build a machine learning model that can predict if an individual's application for a credit card will be accepted.</li>
</ul>
<p>First, loading and viewing the dataset. I find that since this data is confidential, the contributor of the dataset has anonymized the feature names.</p>

In [ ]:
# Import pandas
import pandas as pd, numpy as np

# Load dataset
cc_apps = pd.read_csv( '/content/cc_approvals - Copy (2).data', header=None)

# Inspect data
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


## 2. Inspecting the applications
<p> The features of this dataset have been anonymized to protect the privacy, but <a href="http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html">this blog</a> gives us a pretty good overview of the probable features. The probable features in a typical credit card application are <code>Gender</code>, <code>Age</code>, <code>Debt</code>, <code>Married</code>, <code>BankCustomer</code>, <code>EducationLevel</code>, <code>Ethnicity</code>, <code>YearsEmployed</code>, <code>PriorDefault</code>, <code>Employed</code>, <code>CreditScore</code>, <code>DriversLicense</code>, <code>Citizen</code>, <code>ZipCode</code>, <code>Income</code> and finally the <code>ApprovalStatus</code>. This gives a pretty good starting point, and we can map these features with respect to the columns in the output.   </p>
<p>As you can see from our first glance at the data, the dataset has a mixture of numerical and non-numerical features. This can be fixed with some preprocessing, but before I did that, I did some exploratory analysis to learn about the dataset a bit more to see if there are other dataset issues that need to be fixed.</p>

### **Inspecting the structure, numerical summary, and specific rows of the dataset.**



In [ ]:
# Print summary statistics
cc_apps_description = cc_apps.describe()
print(cc_apps_description)

print('\n')

# Print DataFrame information
cc_apps_info = cc_apps.info()
print(cc_apps_info)

print('\n')

# Inspect missing values in the dataset
cc_apps.tail(17)

               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 no

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
673,?,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0,f,g,00280,364,-
681,b,27.83,1.000,y,p,d,h,3.000,f,f,0,f,g,00176,537,-
682,b,17.08,3.290,u,g,i,v,0.335,f,f,0,t,g,00140,2,-


## 3. Splitting the dataset into train and test sets
<p>Now, I will split our data into train set and test set to prepare the data for two different phases of machine learning modeling: training and testing. Ideally, no information from the test data should be used to preprocess the training data or should be used to direct the training process of a machine learning model. Hence, I first split the data and then preprocess it.</p>
<p>Also, features like <code>DriversLicense</code> and <code>ZipCode</code> are not as important as the other features in the dataset for predicting credit card approvals. We should drop them to design our machine learning model with the best set of features (aka feature selection)

### **Split `cc_apps` into train and test sets**

* Importing **`train_test_split`** from the **`sklearn.model_selection`** module.
* Dropping features 11 and 13 using the **`.drop()`** method.
* Using the **`train_test_split()`** function, split the data into train and test sets with a split ratio of 33% (**`test_size`** argument) and set the **`random_state`** argument to `42`. Assign the train and test DataFrames to the following variables respectively: `cc_apps_train`, `cc_apps_test`.

Note: I also Kept track of the total number of features before and after dropping the features. This often helps with debugging.

I Set **`random_state`** ensures the dataset is split with same sets of instances every time the code is run.



In [ ]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Drop the features 11 and 13
cc_apps = cc_apps.drop(columns=[11, 13])

# Split into train and test sets
cc_apps_train, cc_apps_test = train_test_split(cc_apps, test_size=0.33, random_state=42)

## 4. Handling the missing values (part i)
<p>Splitting the  data, enables me handle some of the issues we identified when inspecting the DataFrame, including:</p>
<ul>
<li>The dataset containing both numeric and non-numeric data (specifically data that are of <code>float64</code>, <code>int64</code> and <code>object</code> types). Specifically, the features 2, 7, 10 and 14 contain numeric values (of types float64, float64, int64 and int64 respectively) and all the other features contain non-numeric values.</li>
<li>The dataset also contains values from several ranges. Some features have a value range of 0 - 28, some have a range of 2 - 67, and some have a range of 1017 - 100000. Apart from these, we can get useful statistical information (like <code>mean</code>, <code>max</code>, and <code>min</code>) about the features that have numerical values. </li>
<li>Finally, the dataset has missing values, which we'll take care of in this task. The missing values in the dataset are labeled with '?', which can be seen in the last cell's output of the second task.</li>
</ul>
<p>Now, I'd temporarily replace these missing value question marks with NaN.</p>

### **Replace the question marks with NaN**

* Import the **`numpy`** library under the alias **`np`**.

* Replace the `'?'`s with NaNs using the **`.replace()`** method in both the train and test sets.




In [ ]:
# Import numpy
import numpy as np

# Replace the '?'s with NaN in the train and test sets
cc_apps_train = cc_apps_train.replace('?',np.nan)
cc_apps_test = cc_apps_test.replace('?', np.nan)

## 5. Handling the missing values (part ii)
<p>I've replaced all the question marks with NaNs. This is going to help us in the next missing value treatment that we are going to perform.</p>
<p>An important question that gets raised here is <em>why are we giving so much importance to missing values</em>? Can't they be just ignored? Ignoring missing values can affect the performance of a machine learning model heavily. While ignoring the missing values our machine learning model may miss out on information about the dataset that may be useful for its training. Then, there are many models which cannot handle missing values implicitly such as Linear Discriminant Analysis (LDA). </p>
<p>So, to avoid this problem, I imputed the missing values with a strategy called mean imputation.</p>

### **Imputing the NaN values with the mean imputation approach.**

* For the numeric columns, I imputed the missing values (NaNs) with pandas method **`.fillna()`**. And I ensured the test set is imputed with the mean values computed from the <ins>training</ins> set.

* I Verified if the **`.fillna()`** method performed as expected by printing the total number of NaNs in each column.


In [ ]:
# Impute the missing values with mean imputation
cc_apps_train.fillna(cc_apps_train.mean(), inplace=True)
cc_apps_test.fillna(cc_apps_train.mean(), inplace=True)

# Count the number of NaNs in the datasets and print the counts to verify
print(cc_apps_train.isnull().sum())
print(cc_apps_test.isnull().sum())

0     8
1     5
2     0
3     6
4     6
5     7
6     7
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64
0     4
1     7
2     0
3     0
4     0
5     2
6     2
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64


<ipython-input-6-3d2ba7b8a823>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cc_apps_train.fillna(cc_apps_train.mean(), inplace=True)
<ipython-input-6-3d2ba7b8a823>:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cc_apps_test.fillna(cc_apps_train.mean(), inplace=True)


## 6. Handling the missing values (part iii)
<p>We have successfully taken care of the missing values present in the numeric columns. There are still some missing values to be imputed for columns 0, 1, 3, 4, 5, 6 and 13. All of these columns contain non-numeric data and this is why the mean imputation strategy would not work here. This needs a different treatment. </p>
<p>On this note, I imputed these missing values with the most frequent values as present in the respective columns. Which is widely accepted practice when it comes to imputing missing values for categorical data in general.</p>

### **Impute the missing values in the <ins>non-numeric</ins> columns**

* Iterate over each column of `cc_apps_train` using a **`for`** loop.

* Check if the data-type of the column is of `object` type by using the **`.dtypes`** keyword.

* Using the **`.fillna()`** method, impute the column's missing values with the most frequent value of that column with the **`.value_counts()`** method and index attribute and assign it to both `cc_apps_train` and `cc_apps_test`. Ensure that the test DataFrame's (`cc_apps_test`) columns are imputed using data from the training DataFrames (`cc_apps_train`).

* Finally, verify if there are any more missing values in the DataFrames that are left to be imputed by printing the total number of NaNs in each column.

The column names of a pandas DataFrame can be accessed using **`.columns`** attribute. The **`.dtypes`** attribute provides the data type. In this part, `object` is the data type that you should be concerned about. The **`.value_counts()`** method returns the frequency distribution of each value in the column, and the **`.index`** attribute can then be used to get the most frequent value.



In [ ]:
# Iterate over each column of cc_apps_train
for col in cc_apps_train.columns:
    # Check if the column is of object type
    if cc_apps_train[col].dtypes == 'object':
        # Impute with the most frequent value
        cc_apps_train = cc_apps_train.fillna(cc_apps_train[col].value_counts().index[0])
        cc_apps_test = cc_apps_test.fillna(cc_apps_train[col].value_counts().index[0])

# Count the number of NaNs in the dataset and print the counts to verify
print(cc_apps_train.isna().sum())
print(cc_apps_test.isna().sum())


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64


## 7. Preprocessing the data (part i)
<p>The missing values are now successfully handled.</p>
<p>There is still some minor but essential data preprocessing needed before we proceed towards building our machine learning model. We are going to divide these remaining preprocessing steps into two main tasks:</p>
<ol>
<li>Convert the non-numeric data into numeric.</li>
<li>Scale the feature values to a uniform range.</li>
</ol>
<p>First, I converted all the non-numeric values into numeric ones. I did this because not only it results in a faster computation but also many machine learning models (like XGBoost) (and especially the ones developed using scikit-learn) require the data to be in a strictly numeric format. I did this by using the <code>get_dummies()</code> method from pandas.</p>

### **Converting the non-numeric values to numeric**

* Use the **`.get_dummies()`** method on `cc_apps_train` and then use it on `cc_apps_test`. Store the outputs in `cc_apps_train` and `cc_apps_test` variables respectively.

* Use the **`.reindex()`** method on `cc_apps_test` using the columns from `cc_apps_train`. Use `0` to fill the columns that aren't present in the reindexed `cc_apps_test`.

The last reindexing step is used for discarding any new categorical feature that'd appear in the test data.



In [ ]:
# Convert the categorical features in the train and test sets independently
cc_apps_train = pd.get_dummies(cc_apps_train)
cc_apps_test = pd.get_dummies(cc_apps_test)

# Reindex the columns of the test set aligning with the train set
cc_apps_test = cc_apps_test.reindex(columns=cc_apps_train.columns, fill_value=0)

## 8. Preprocessing the data (part ii)
<p>I am left with one final preprocessing step of scaling before we can fit a machine learning model to the data. </p>
<p>To understand what these scaled values mean in the real world. After some level of exploration, I decided to use <code>CreditScore</code> as an example. The credit score of a person is their creditworthiness based on their credit history. The higher this number, the more financially trustworthy a person is considered to be. So, a <code>CreditScore</code> of 1 is the highest since we're rescaling all the values to the range of 0-1.</p>

### **Rescaling the features of the data**

* Import the **`MinMaxScaler`** class from the **`sklearn.preprocessing`** module.

* Segregate the features and labels into `X_train`, `y_train`, `X_test`, and `y_test` variables respectively. You can use **`.iloc`** for this purpose.

* Instantiate **`MinMaxScaler`** class in a variable called `scaler` with the **`feature_range`** parameter set to `(0,1)`.

* Fit the `scaler` to `X_train` and transform the data, assigning the result to `rescaledX_train`.
* Use the `scaler` to transform `X_test`, assigning the result to `rescaledX_test`.

When a dataset has varying ranges as in this credit card approvals dataset, one a small change in a particular feature may not have a significant effect on the other feature, which can cause a lot of problems when predictive modeling.



In [ ]:
# Import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# Segregate features and labels into separate variables
X_train, y_train = cc_apps_train.iloc[:, :-1].values, cc_apps_train.iloc[:, -1].values
X_test, y_test = cc_apps_test.iloc[:, :-1].values, cc_apps_test.iloc[:, -1].values

# Instantiate MinMaxScaler and use it to rescale X_train and X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

## 9. Fitting a logistic regression model to the train set
<p>Essentially, predicting if a credit card application will be approved or not is a <a href="https://en.wikipedia.org/wiki/Statistical_classification">classification</a> task. According to UCI, our dataset contains more instances that correspond to "Denied" status than instances corresponding to "Approved" status. Specifically, out of 690 instances, there are 383 (55.5%) applications that got denied and 307 (44.5%) applications that got approved. </p>
<p>This gives us a benchmark. A good machine learning model should be able to accurately predict the status of the applications with respect to these statistics.</p>
<p>The question to ask is: <em>are the features that affect the credit card approval decision process correlated with each other?</em>

### **Fit a `LogisticRegression` classifier with `rescaledX_train` and `y_train`**

* Import **`LogisticRegression`** from the **`sklearn.linear_model`** module.
* Instantiate **`LogisticRegression`** into a variable named `logreg` with default values.
* Fit `rescaledX_train` and `y_train` to `logreg` using the **`.fit()`** method.



In [ ]:
# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

# Instantiate a LogisticRegression classifier with default parameter values
logreg = LogisticRegression()

# Fit logreg to the train set
logreg.fit(rescaledX_train, y_train)

LogisticRegression()

## 10. Making predictions and evaluating performance
<p>The question is how well does the model perform? </p>
<p>I evaluated the model on the test set with respect to <a href="https://developers.google.com/machine-learning/crash-course/classification/accuracy">classification accuracy</a>. I also had to take a look the model's <a href="http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/">confusion matrix</a>. In the case of predicting credit card applications, it is important to see if our machine learning model is equally capable of predicting approved and denied status, in line with the frequency of these labels in our original dataset. If the model is not performing well in this aspect, then it might end up approving the application that should have been approved. The confusion matrix helps us to view our model's performance from these aspects.  </p>

### **Make predictions and evaluate performance**

* Import **`confusion_matrix()`** from **`sklearn.metrics`** module.

* Use **`.predict()`** on `rescaledX_test` (which contains instances of the dataset that `logreg` has not seen until now) and store the predictions in a variable named `y_pred`.

* Print the accuracy score of `logreg` using the **`.score()`**. Don't forget to pass `rescaledX_test` and `y_test` to the **`.score()`** method.

* Call **`confusion_matrix()`** with `y_test` and `y_pred` to print the confusion matrix.



In [ ]:
# Import confusion_matrix
from sklearn.metrics import confusion_matrix

# Use logreg to predict instances from the test set and store it
y_pred = logreg.predict(rescaledX_test)

# Get the accuracy score of logreg model and print it
print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test, y_test))

# Print the confusion matrix of the logreg model
confusion_matrix(y_test, y_pred)

Accuracy of logistic regression classifier:  1.0


array([[103,   0],
       [  0, 125]])

## 11. Grid searching and making the model perform better
<p>The model was pretty good! In fact it was able to yield an accuracy score of 100%.</p>
<p>For the confusion matrix, the first element of the of the first row of the confusion matrix denotes the true negatives meaning the number of negative instances (denied applications) predicted by the model correctly. And the last element of the second row of the confusion matrix denotes the true positives meaning the number of positive instances (approved applications) predicted by the model correctly.</p>
<p>if I hadn't got a perfect score what's to be done?. I can perform a <a href="https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/">grid search</a> of the model parameters to improve the model's ability to predict credit card approvals.</p>
<p><a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">scikit-learn's implementation of logistic regression</a> consists of different hyperparameters but I grid search over the following two:</p>
<ul>
<li>tol</li>
<li>max_iter</li>
</ul>

### **Define the grid of parameter values for which grid searching is to be performed**

* Import **`GridSearchCV`** from the **`sklearn.model_selection`** module.

* Define the grid of values for **`tol`** and **`max_iter`** parameters into `tol` and `max_iter` lists respectively.

* For `tol`, define the list with values `0.01, 0.001, 0.0001`. For `max_iter`, define the list with values `100, 150, 200`.

* Using the **`.dict()`** method, create a dictionary where `tol` and `max_iter` are keys, and the lists of their values are the corresponding values. Name this dictionary as `param_grid`.

Grid search can be very exhaustive if the model is very complex and the dataset is extremely large. Luckily, that is not the case for this project.

In [ ]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Define the grid of values for tol and max_iter
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]

# Create a dictionary where tol and max_iter are keys and the lists of their values are corresponding values
param_grid = {'tol': tol, 'max_iter': max_iter}

## 12. Finding the best performing model
<p>We have defined the grid of hyperparameter values and converted them into a single dictionary format which <code>GridSearchCV()</code> expects as one of its parameters. Now, we will begin the grid search to see which values perform best.</p>
<p>We will instantiate <code>GridSearchCV()</code> with our earlier <code>logreg</code> model with all the data we have. We will also instruct <code>GridSearchCV()</code> to perform a <a href="https://www.dataschool.io/machine-learning-with-scikit-learn/">cross-validation</a> of five folds.</p>
<p>We'll end the notebook by storing the best-achieved score and the respective best parameters.</p>
<p>While building this credit card predictor, we tackled some of the most widely-known preprocessing steps such as <strong>scaling</strong>, <strong>label encoding</strong>, and <strong>missing value imputation</strong>. We finished with some <strong>machine learning</strong> to predict if a person's application for a credit card would get approved or not given some information about that person.</p>

### **Find the best score and best parameters for the model using grid search**

* Instantiate **`GridSearchCV()`** with the attributes set as
    * **`estimator=logreg`
    * **`param_grid=param_grid`**
    * **`cv=5`**
    and store this instance in `grid_model` variable.
    
* Fit `rescaledX_train` and `y_train` to `grid_model` and store the results in `grid_model_result`.

* Call the **`.best_score_`** and **`.best_params_`** attributes on the `grid_model_result` variable, then print both.

* Extract the best model from `grid_model_result` and evaluate it on the test set (`rescaledX_test, y_test`).


In [ ]:
# Instantiate GridSearchCV with the required parameters
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Fit grid_model to the data
grid_model_result = grid_model.fit(rescaledX_train, y_train)

# Summarize results
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

# Extract the best model and evaluate it on the test set
best_model = grid_model_result.best_estimator_
print("Accuracy of logistic regression classifier: ", best_model.score(rescaledX_test, y_test))

Best: 1.000000 using {'max_iter': 100, 'tol': 0.01}
Accuracy of logistic regression classifier:  1.0
